<a href="https://colab.research.google.com/github/victoralcantara75/CNN-FLOWER-CLASSIFY/blob/master/CNN_FLOWER_CLASSIFY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CNN PARA CLASSIFICAÇÃO DE FLORES
## SIN493 - DEEP LEARNING PARA VISÃO COMPUTACIONAL

##PYTORCH E CUDA

In [0]:
#http://pytorch.org/

from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [49]:
import numpy as np

# verificando se o CUDA está disponível
GPUavailable = torch.cuda.is_available()

if GPUavailable:
    print('Treinamento em GPU!')
else:
    print('Treinamento em CPU!')

Treinamento em GPU!


##DATASET

In [50]:
from torchvision import datasets
import torchvision.transforms as transforms

!wget -cq https://github.com/udacity/pytorch_challenge/raw/master/cat_to_name.json
!wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
!unzip -qq flower_data.zip

replace flower_data/valid/61/image_06296.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace flower_data/valid/61/image_06293.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
diretorio_dados = 'flower_data'
diretorio_treino = diretorio_dados + '/train'
diretorio_validacao = diretorio_dados + '/valid'

In [0]:
# APLICANDO AS TRANSFORMACOES NECESSARIAS E O DATA AUGMENTATION

transformacao = {'train': 
                   transforms.Compose([ 
                   transforms.RandomRotation(10),      #ROTAÇÃO DAS IMAGENS
                   transforms.RandomResizedCrop(224),  #INPUT SIZE 224X224
                   transforms.RandomHorizontalFlip(),  #ESPELHAMENTO
                   transforms.ToTensor(),
                   transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ]),
                'valid': 
                   transforms.Compose([ 
                   transforms.Resize(256),
                   transforms.CenterCrop(224),
                   transforms.ToTensor(),
                   transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ])
}

In [63]:
import os

# image_datasets = { x: datasets.ImageFolder(os.path.join(diretorio_dados, x), transformacao[x])
#     for x in ['train', 'valid']
# }

# batch_size =4 

# dataloaders = { x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4)
#     for x in ['train', 'valid']
# }


dados_treino = datasets.ImageFolder(diretorio_treino, transformacao)
dados_validacao = datasets.ImageFolder(diretorio_validacao, transformacao)

print('Numero de imagens de treino: ', len(dados_treino))
print('Numero de imagens de teste: ', len(dados_validacao))

Numero de imagens de treino:  6552
Numero de imagens de teste:  818


In [0]:
# Data Loaders e Visualização

batch_size = 20
num_workers=0

# dataloaders
treino_loader = torch.utils.data.DataLoader(dados_treino, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
validacao_loader = torch.utils.data.DataLoader(dados_validacao, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)

In [0]:
#MAPEANDO OS LABELS
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [72]:
# obtendo um batch de imagens de treinamento
dataiter = iter(treino_loader)
images, labels = dataiter.next()
images = images.numpy() # para exibir as imagens converter para numpy

# plotar as imagens no batch, com os rótulos correspondentes
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    plt.imshow(np.transpose(images[idx], (1, 2, 0)))
    ax.set_title(classes[labels[idx]])

TypeError: ignored

##TREINO E CLASSIFICACAO VGG

In [0]:
import torchvision.models as models

#criando o modelo de vgg 16 pre treinada
VGG = models.vgg16(pretrained=True)
#visualizacao da arquitetura utilizada vgg16
VGG

# movendo os tensors para GPU se o CUDA estiver disponível
if GPUavailable:
    VGG.cuda()


In [0]:
# Congelar o treinamento para todas as camadas de características
for param in VGG.features.parameters():
    param.requires_grad = False

In [68]:
import torch.nn as nn

n_inputs = VGG.classifier[6].in_features

# adicionar a última camada linear(n_inputs -> 5 classes de flores)
# novas camadas automaticamente tem requires_grad = True
last_layer = nn.Linear(n_inputs, 102)

VGG.classifier[6] = last_layer

# se a GPU está disponível, mover o modelo para GPU
if GPUavailable:
    VGG.cuda()

# verifique se sua última camada produz o número esperado de saídas
print(VGG.classifier[6].out_features)
#print(vgg16)

102


In [0]:
import torch.nn as nn
import torch.optim as optim

# Especificando a loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# Definindo o Otimizador
optimizer = optim.SGD(VGG.parameters(), lr=0.01)

In [70]:
#TREINAMENTO 

epocas = 2

for epocas in range (1, epocas+1):

  treino_loss = 0.0
  validacao_loss = 0.0
  
  VGG.train()
  for batch_i, (data, target) in enumerate(treino_loader):
        # mover os tensor para GPU se o CUDA estiver disponível
        if GPUavailable:
            data, target = data.cuda(), target.cuda()
        # limpar os gradientes de todas as variáveis otimizadas
        optimizer.zero_grad()
        # forward: calcula as saídas previstas passando as entradas ao modelo
        output = VGG(data)
        # calcular o decaimento do batch
        loss = criterion(output, target)
        # backward: computar o gradiente da perda (loss) em relação aos parâmetros do modelo
        loss.backward()
        # executar uma única etapa de otimização (atualização de parâmetro)
        optimizer.step()
        # atualizar o loss do treinamento 
        treino_loss += loss.item()
        
        if batch_i % 20 == 19:    # print do treinamento loss especificando o número de mini-batches
            print('Época %d, Batch %d loss: %.16f' %
                  (epoch, batch_i + 1, train_loss / 20))
            train_loss = 0.0
        
 

TypeError: ignored